# Data Selection 
1. Select only the columns we need for running `prophet`.
2. Select sales data including and after 2018/10/01, in order to accommodate constraints on computational resources and time.

In [2]:
import os

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
from src.data_helpers import load_lookup_tables

In [2]:
dept_lookup = pd.read_csv("data/dept_lookup.txt", sep="|")

# Making sure that np.int16 can represent the largest DEPARTMENT_ID
dept_lookup.DEPARTMENT_ID.max(), np.int16(dept_lookup.DEPARTMENT_ID.max())

(99, 99)

In [3]:
store_lookup = pd.read_csv("data/store_lookup.txt", sep="|")

# Making sure that np.int16 can represent the largest LOC_ID
store_lookup.LOC_ID.max(), np.int16(store_lookup.LOC_ID.max())

(7899.0, 7899)

In [4]:
bm_sales_col_dtypes = {
    "LOC_ID": np.int16,
    "DEPARTMENT_ID": np.int16,
    "ACTUAL_SALES": np.float16
}

In [7]:
%%time

bm_sales_df = []

bm_sales_csviter = pd.read_csv(
    "data/day_bm_loc_dep_sales_201801_202102.txt.gz",
    sep="|",
    dtype=bm_sales_col_dtypes,
    usecols=["DAY_ID", "LOC_ID", "DEPARTMENT_ID", "ACTUAL_SALES"],
    parse_dates=["DAY_ID"],
    chunksize=10000,
)


for chunk in bm_sales_csviter:
    bm_sales_df.append(chunk.query("`DAY_ID` >= '2018-10-01'"))
    
bm_sales_df = pd.concat(bm_sales_df, ignore_index=True)

CPU times: user 9min 9s, sys: 34.5 s, total: 9min 44s
Wall time: 9min 39s


In [8]:
bm_sales_df.head(10)

,DAY_ID,LOC_ID,DEPARTMENT_ID,ACTUAL_SALES
0,2020-12-03,2768,13,135.500000
1,2018-12-09,3811,20,33.031250
2,2019-04-28,739,21,23.984375
3,2019-11-26,1179,50,15.882812
4,2019-06-25,2433,58,13.992188
5,2020-05-02,2402,21,29.984375
6,2020-12-21,917,79,5.988281
7,2020-04-29,201,11,33.968750
8,2019-01-19,1929,82,20.984375
9,2019-04-18,725,22,16.984375


In [9]:
# Now we have a 2GB dataframe
bm_sales_df.memory_usage().sum() / 1e6

2198.322072

In [ ]:
# Persist to disk for later 
bm_sales_df.to_pickle("data/bm_sales_df.pickle")

## Save each department as separate files 

Formatted for `prophet` usage

In [3]:
bm_sales_df = pd.read_pickle("data/bm_sales_df.pickle")

loc_table, dept_table = load_lookup_tables(
    os.path.join("data", "store_lookup.txt"),
    os.path.join("data", "dept_lookup.txt")
)

In [22]:
for dept_id in dept_table.DEPARTMENT_ID.unique():
    # print(dept_id, dept_table.query("`DEPARTMENT_ID` == @dept_id").iloc[0, 1])
    (
        bm_sales_df
        .query("`DEPARTMENT_ID` == @dept_id")
        .astype({"ACTUAL_SALES": np.float32})  # upcast, otherwise overflow 
        .groupby("DAY_ID")["ACTUAL_SALES"].sum()
        .reset_index()
        .rename(columns={"DAY_ID": "ds", "ACTUAL_SALES": "y"})
        .to_pickle(os.path.join("data", "per_dept_daily_sales", f"dept_{dept_id}_daily.pickle"))
    )